### 대규모 데이터 처리를 위한 sparse matrix 사용

 대용량의 데이터를 처리하기 위해서는 메모리의 한계가 있다. 희박행렬(sparse matrix)는 인덱스와 값 만을 저장해 일반적인 matrix에 비해 저장공간이 절약되는 정도가 커진다.  
  그러나 단점도 존재하는데 데이터 처리의 overhead cost가 크다는 점이다.  
  python에서도 scipy클래스의 하나로서 이와 같은 sparse matrix를 구현해 놓았다.  
  여기서는 csr_mtrix( Compressed Sparse Row foramt : 효율적인 row 슬라이싱)를 사용하기로 한다.

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

In [2]:
ratings = {'user_id': [1,2,4], 
     'movie_id': [2,3,7], 
     'rating': [4,3,1]}
ratings = pd.DataFrame(ratings)

In [3]:
ratings

,user_id,movie_id,rating
0,1,2,4
1,2,3,3
2,4,7,1


- Pandas pivot을 이용해서 full matrix로 변환하는 경우

In [4]:
rating_matrix = ratings.pivot(index = 'user_id', columns ='movie_id', 
                values = 'rating').fillna(0)
full_matrix1 = np.array(rating_matrix)
print(full_matrix1)

[[4. 0. 0.]
 [0. 3. 0.]
 [0. 0. 1.]]


- Sparse matrix를 이용해서 full matrix로 변환하는 경우

In [5]:
data = np.array(ratings['rating'])
row_indices = np.array(ratings['user_id'])
col_indices = np.array(ratings['movie_id'])
rating_matrix = csr_matrix((data, (row_indices, col_indices)), 
                dtype=int)
print(rating_matrix)

  (1, 2)	4
  (2, 3)	3
  (4, 7)	1


In [6]:
full_matrix2 = rating_matrix.toarray()
print(full_matrix2)

[[0 0 0 0 0 0 0 0]
 [0 0 4 0 0 0 0 0]
 [0 0 0 3 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]]


- sparse matrix 계산  
     아래에서 보는 것처럼 sparse matrix도 일반 matrix처럼 연산을 할 수 있다.

In [7]:
print(rating_matrix * 2)

  (1, 2)	8
  (2, 3)	6
  (4, 7)	2


- sparse matrix를 추천 알고리즘에 적용하기

- 아래 코드는 movielens 20m 데이터로 full_matrix로 만드는 코드인데 메모리에 담기에 데이터가 너무 크므로 오류가 발생한다.  
    그러므로 위에서 사용한 sparsematrix를 사용하기로 하자

In [8]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('ratings-20m.csv', names=r_cols, sep=',',encoding='latin-1') # 20M data

In [9]:
R_temp = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

ValueError: Unstacked DataFrame is too big, causing int32 overflow

In [11]:
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)            # timestamp 제거

# train test 분리
from sklearn.utils import shuffle
TRAIN_SIZE = 0.75
ratings = shuffle(ratings, random_state=1)
cutoff = int(TRAIN_SIZE * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

- sparse matrix 사용

In [12]:
from scipy.sparse import csr_matrix
data = np.array(ratings['rating'])
row_indices = np.array(ratings['user_id'])
col_indices = np.array(ratings['movie_id'])
ratings = csr_matrix((data, (row_indices, col_indices)), dtype=int)

- train과 test데이터에 적용하기위한 MF 클래스 만들기

In [13]:
class NEW_MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        self.R = ratings
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose

    # train set의 RMSE 계산
    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))

    # Ratings for user i and item j
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    # Test set을 선정
    def set_test(self, ratings_test):
        test_set = []
        for i in range(len(ratings_test)):              # test 데이터에 있는 각 데이터에 대해서
            x, y, z = ratings_test.iloc[i]              # user_id, item_id, rating 가져오기
            test_set.append([x, y, z])
            self.R[x, y] = 0                            # Setting test set ratings to 0
        self.test_set = test_set
        return test_set                                 # Return test set

    # Test set의 RMSE 계산
    def test_rmse(self):
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0], one_set[1])
            error += pow(one_set[2] - predicted, 2)
        return np.sqrt(error/len(self.test_set))

    # Training 하면서 test set의 정확도를 계산
    def test(self):
        # Initializing user-feature and item-feature matrix
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        # List of training samples
        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]

        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1 = self.rmse()
            rmse2 = self.test_rmse()
            training_process.append((i+1, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.4f ; Test RMSE = %.4f" % (i+1, rmse1, rmse2))
        return training_process

    # 주어진 사용자아이디와 영화 아이디로 구한 평점예측
    def get_one_prediction(self, user_id, item_id):
        return self.get_prediction(user_id, item_id)

In [14]:
R_temp = ratings.copy()

- 아래의 코드로 실행할 수 있으나 48시간이 걸릴정도로 상당히 오랜 시간이 걸린다.
    오랜 시간이 걸렸지만 sparse matrix가 아닌 numpy array로는 불가능한 매우 큰 규모의 데이터를 분석할 수 있다는 장점이 있다.

In [15]:
mf = NEW_MF(R_temp, K=200, alpha=0.001, beta=0.02, iterations=250, verbose=True)
test_set = mf.set_test(ratings_test)
result = mf.test()

KeyboardInterrupt: 

### 추천 시스템 구축에서의 이슈

- 신규 사용자와 아이템

협업 필터링은 기본적으로는 데이터가 없는 사용자나 아이템에 대해서는 추천을 만들어낼 수 없다. 그래서 데이터가 없는 사용자나  
아이템에 대해서는 추천을 만들어낼 수 없다. 그래서 데이터가 없는 사용자나 아이템에 대해서 어떻게 할지를 결정해야 한다.  
가장 많이 사용하는 방식은 best seller를 추천하는 것이다. 그리고 사용자들에게 제시되는 추천 리스트에 신규 아이템을 임의로 한 번씩  
넣어서 사용자의 클릭과 평가를 유도하는 방법을 사용할 수도 있다.

- 확장성

실제 데이터는 수백만 명의 고객과 수천만 가지의 아이템을 사용해야 하기 때문에 효율적인 계산이 매우 중요하다.  
그러므로 계산의 효율성을 위해 Spark와 같은 병렬처리 기술로 대규모 데이터를 빠르게 연산 및 처리하는 능력을 갖추는 것이 필요하다.  
또한 배치(batch)방식으로 시스템이 한가할 때 미리 해놓고 그 결과를 저장하는 방법을 사용할 수 있다.  
앞에서 배운대로 Item-based CF가 계산량이 적으므로 더 유리하고 샘플링을 통해 대표적인 사용자와 아이템을 골라서 모델을 학습시키고  
사용하는 방법도 있다.

- 추천의 활용

추천으로 리스트를 제공할 수 있지만 실제로 고객의 예상 선호도를 기준으로 정렬해서 검색 결과의 정확성을 높이는 등의 다양한 방법으로  
활용할 수 있다.

- 사용자의 간접 평가 데이터 확보

 대체로 고객들은 리턴이 크지 않거나 불만족스럽지 않다면 평가하지 않는 경향이 있다.  
 그러므로, 온라인 쇼핑몰에서 어떤 제품 페이지에 사용자가 몇번이나 방문했고 얼마나 시간을 보냈는가 하는 행동로그데이터가  
 고객의 선호도를 측정하는데 효과적으로 사용할 수 있다.